In [1]:
import optuna
import json
from sklearn.metrics import (
    confusion_matrix, precision_score, recall_score, f1_score, accuracy_score, balanced_accuracy_score
)
import numpy as np

In [ ]:
# --- Almacenamiento de resultados ---
all_results = []
best_by_model = {}

In [ ]:
# Para secuenciales
train_loader_seq = DataLoader(VideoDataset(vids_training, labels_training), batch_size=32, shuffle=True)
val_loader_seq   = DataLoader(VideoDataset(vids_val, labels_val), batch_size=32, shuffle=False)

# Para GCN
train_loader_graph = GeoDataLoader(VideoGraphDataset(vids_training, labels_training), batch_size=32, shuffle=True)
val_loader_graph   = GeoDataLoader(VideoGraphDataset(vids_val, labels_val), batch_size=32, shuffle=False)

In [ ]:
def compute_metrics(y_true, y_pred, labels):
    cm = confusion_matrix(y_true, y_pred, labels=labels)
    tn = np.diag(cm)  # diagonal principal son los verdaderos positivos por clase
    FP = cm.sum(axis=0) - tn
    FN = cm.sum(axis=1) - tn
    TP = tn
    TN = cm.sum() - (FP + FN + TP)
    
    # Evitar divisiones por cero
    eps = 1e-8
    specificity = np.mean(TN / (TN + FP + eps))
    precision = precision_score(y_true, y_pred, average='macro', zero_division=0)
    recall = recall_score(y_true, y_pred, average='macro', zero_division=0)
    f1 = f1_score(y_true, y_pred, average='macro', zero_division=0)
    acc = accuracy_score(y_true, y_pred)
    bal_acc = balanced_accuracy_score(y_true, y_pred)

    return {
        "accuracy": acc,
        "precision_macro": precision,
        "recall_macro": recall,
        "specificity_macro": specificity,
        "f1_macro": f1,
        "balanced_accuracy": bal_acc,
    }

In [ ]:
def train_and_validate(model, lr, train_loader, val_loader, device, is_graph=False):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    epochs = 100

    for _ in range(epochs):
        model.train()
        for batch in train_loader:
            optimizer.zero_grad()
            if is_graph:
                batch = batch.to(device)
                outputs = model(batch.x, batch.edge_index, batch.batch)
                loss = criterion(outputs, batch.y)
            else:
                X, y = batch
                X, y = X.to(device), y.to(device)
                outputs = model(X)
                loss = criterion(outputs, y)
            loss.backward()
            optimizer.step()

    # --- Validación ---
    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for batch in val_loader:
            if is_graph:
                batch = batch.to(device)
                outputs = model(batch.x, batch.edge_index, batch.batch)
                preds = outputs.argmax(dim=1).cpu().numpy()
                y_true.extend(batch.y.cpu().numpy())
                y_pred.extend(preds)
            else:
                X, y = batch
                X, y = X.to(device), y.to(device)
                outputs = model(X)
                preds = outputs.argmax(dim=1).cpu().numpy()
                y_true.extend(y.cpu().numpy())
                y_pred.extend(preds)

    metrics = compute_metrics(y_true, y_pred, labels=list(range(4)))
    return metrics

In [ ]:
# --- Función objective global ---
def objective(trial):
    model_name = trial.suggest_categorical("model", ["Transformer", "GCN", "MS-TCN", "TGU", "SAN", "LSTM"])
    lr = trial.suggest_loguniform("lr", 1e-5, 1e-2)
    hidden = trial.suggest_categorical("hidden", [64, 128, 256])
    dropout = trial.suggest_float("dropout", 0.1, 0.5)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    if model_name == "Transformer":
        model = TransformerBaseline(d_model=hidden, dropout=dropout).to(device)
        is_graph = False
        train_loader, val_loader = train_loader_seq, val_loader_seq

    elif model_name == "GCN":
        model = GCNBaseline(hidden_channels=hidden).to(device)
        is_graph = True
        train_loader, val_loader = train_loader_graph, val_loader_graph

    elif model_name == "MS-TCN":
        model = MSTCNBaseline().to(device)
        is_graph = False
        train_loader, val_loader = train_loader_seq, val_loader_seq

    elif model_name == "TGU":
        model = TGUBaseline(hidden=hidden).to(device)
        is_graph = False
        train_loader, val_loader = train_loader_seq, val_loader_seq

    elif model_name == "SAN":
        model = SANBaseline(hidden=hidden).to(device)
        is_graph = False
        train_loader, val_loader = train_loader_seq, val_loader_seq

    elif model_name == "LSTM":
        model = LSTMBaseline(hidden1=hidden, hidden2=hidden, hidden3=hidden//2).to(device)
        is_graph = False
        train_loader, val_loader = train_loader_seq, val_loader_seq

    metrics = train_and_validate(model, lr, train_loader, val_loader, device, is_graph)
    val_acc = metrics["accuracy"]

    # Guardar resultados
    result = {
        "trial": trial.number,
        "model": model_name,
        "lr": lr,
        "hidden": hidden,
        "dropout": dropout,
        "metrics": metrics
    }
    all_results.append(result)

    # Mejor por modelo
    if model_name not in best_by_model or val_acc > best_by_model[model_name]["metrics"]["accuracy"]:
        best_by_model[model_name] = result

    return val_acc

In [ ]:
"""
# --- Ejecución ---
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)

# --- Guardar resultados ---
with open("optuna_all_results.json", "w") as f:
    json.dump(all_results, f, indent=4)
3
with open("optuna_best_models.json", "w") as f:
    json.dump(best_by_model, f, indent=4)

print("\n✅ Resultados guardados en:")
print("  • optuna_all_results.json")
print("  • optuna_best_models.json")
print("\n🏆 Mejor modelo global:")
print(study.best_params)
print(f"Accuracy de validación: {study.best_value:.4f}")
"""